In [9]:
import pandas as pd

## Institutes

In [10]:
df = pd.read_csv('../outputs/clean/institutes.csv')
df.head()

,id,nome,tipo,posti_regolamentari,posti_non_disponibili,posti_occupati,posti_aggiornati_al,asl,nome_responsabile_asl,cognome_responsabile_asl,...,cognome_direttore,ruolo_direttore,personale_polizia_effettivi,personale_polizia_previsti,personale_amministrativi_effettivi,personale_amministrativi_previsti,personale_educatori_effettivi,personale_educatori_previsti,personale_polizia_aggiornato_a,personale_amministrativo_aggiornato_al
0,MII179988,Reggio Calabria Arghillà,Casa circondariale,294,0,319,2024-10-05,5 Reggio Calabria,Luciano,Lucania,...,Stendardo,Titolare,131,164,12,32,5,8,2024-07-31,2024-07-31
1,MII172610,Brescia Verziano,Casa di reclusione,71,0,119,2024-10-05,Spedali Civili di Brescia,Luigi,Leone,...,NaN,NaN,79,95,2,0,1,0,2024-07-31,2024-07-31
2,MII172320,Busto Arsizio,Casa circondariale,240,14,431,2024-10-05,Busto Arsizio,Ezia,Iorio,...,Pitaniello,Titolare,193,190,16,21,5,5,2024-07-31,2024-07-31
3,MII173712,Como,Casa circondariale,226,1,426,2024-10-05,Como,Giuseppe,Carrano,...,Rinaldi,Titolare,208,216,18,23,6,6,2024-07-31,2024-07-31
4,MII173747,Cremona,Casa circondariale,394,10,586,2024-10-05,CREMONA,Rossano,Botto,...,Padula,Titolare,188,202,15,23,5,6,2024-07-31,2024-07-31


## Most recent data

In [11]:
# Get the most recent update
df['posti_aggiornati_al'] = pd.to_datetime(df['posti_aggiornati_al'], format='%Y-%m-%d')
df_most_recent = df.loc[df.groupby('id')['posti_aggiornati_al'].idxmax()]

# Adding columns for places available and overcrowding index
df_most_recent['posti_reali'] = df_most_recent['posti_regolamentari'] - df_most_recent['posti_non_disponibili']
df_most_recent['tasso_sovraffollamento'] = round(df_most_recent['posti_occupati'] / df_most_recent['posti_reali'],2)
df_most_recent.head()

,id,nome,tipo,posti_regolamentari,posti_non_disponibili,posti_occupati,posti_aggiornati_al,asl,nome_responsabile_asl,cognome_responsabile_asl,...,personale_polizia_effettivi,personale_polizia_previsti,personale_amministrativi_effettivi,personale_amministrativi_previsti,personale_educatori_effettivi,personale_educatori_previsti,personale_polizia_aggiornato_a,personale_amministrativo_aggiornato_al,posti_reali,tasso_sovraffollamento
369,MII152284,Genova Marassi,Casa circondariale,535,0,676,2024-10-06,ASL,Maurizio,Cechini,...,357,336,26,35,10,10,NaN,NaN,535,1.26
232,MII157783,Alba,Casa di reclusione - Giuseppe Montalto,138,89,38,2024-10-06,CN2 Alba-Bra,Paolo,Lo Russo,...,88,107,10,14,3,3,NaN,NaN,49,0.78
302,MII158895,Milano San Vittore,Casa circondariale - Francesco Di Cataldo,700,254,1016,2024-10-06,Milano,Ruggiero,Giuliani,...,583,655,35,45,13,14,NaN,NaN,446,2.28
337,MII158901,Belluno,Casa circondariale,89,7,105,2024-10-06,n.1 Belluno,Marco,Cristofoletti,...,81,85,8,14,2,2,NaN,NaN,82,1.28
290,MII158910,Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,4,387,2024-10-06,ASP 1 AGRIGENTO,GIUSEPPE,MANCUSO,...,217,197,23,23,5,5,NaN,NaN,279,1.39


In [12]:
df_most_recent.value_counts(['posti_aggiornati_al'])

posti_aggiornati_al
2024-10-06             190
dtype: int64

In [13]:
# Adding institutes' information for mapping the institutes
# Read institutes' csv

df_info = pd.read_csv('../outputs/clean/institutes_info.csv')
df_info = df_info.rename(columns={'id_istituto': 'id'})

merged_df = pd.merge(df_most_recent, df_info, on='id')

# Adding additional columns
merged_df['posti_reali'] = merged_df['posti_regolamentari'] - merged_df['posti_non_disponibili']

merged_df['tasso_affollamento'] = (merged_df['posti_occupati'] / merged_df['posti_reali']).round(2)

# Keep only relevant columns
df_filtered = merged_df[
  [
    'id',
    'nome',
    'tasso_affollamento',
    'indirizzo',
    'tipo',
   'posti_regolamentari',
    'posti_non_disponibili',
    'posti_reali',
    'posti_occupati',
    'posti_aggiornati_al',
    'personale_polizia_previsti',
    'personale_polizia_effettivi',
    'personale_polizia_aggiornato_a',
    'personale_amministrativi_previsti',
    'personale_amministrativi_effettivi',
    'personale_amministrativo_aggiornato_al',
    'latitudine',
    'longitude'
    ]
    ]

df_filtered.head()

,id,nome,tasso_affollamento,indirizzo,tipo,posti_regolamentari,posti_non_disponibili,posti_reali,posti_occupati,posti_aggiornati_al,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,personale_amministrativi_previsti,personale_amministrativi_effettivi,personale_amministrativo_aggiornato_al,latitudine,longitude
0,MII152284,Genova Marassi,1.26,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,676,2024-10-06,336,357,NaN,35,26,NaN,44.417913,8.951100
1,MII157783,Alba,0.78,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,38,2024-10-06,107,88,NaN,14,10,NaN,44.691542,8.025636
2,MII158895,Milano San Vittore,2.28,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,700,254,446,1016,2024-10-06,655,583,NaN,45,35,NaN,45.461598,9.166399
3,MII158901,Belluno,1.28,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,105,2024-10-06,85,81,NaN,14,8,NaN,46.139738,12.219297
4,MII158910,Agrigento,1.39,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,4,279,387,2024-10-06,197,217,NaN,23,23,NaN,37.319422,13.617396


In [14]:
df_filtered.head()

,id,nome,tasso_affollamento,indirizzo,tipo,posti_regolamentari,posti_non_disponibili,posti_reali,posti_occupati,posti_aggiornati_al,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,personale_amministrativi_previsti,personale_amministrativi_effettivi,personale_amministrativo_aggiornato_al,latitudine,longitude
0,MII152284,Genova Marassi,1.26,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,676,2024-10-06,336,357,NaN,35,26,NaN,44.417913,8.951100
1,MII157783,Alba,0.78,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,38,2024-10-06,107,88,NaN,14,10,NaN,44.691542,8.025636
2,MII158895,Milano San Vittore,2.28,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,700,254,446,1016,2024-10-06,655,583,NaN,45,35,NaN,45.461598,9.166399
3,MII158901,Belluno,1.28,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,105,2024-10-06,85,81,NaN,14,8,NaN,46.139738,12.219297
4,MII158910,Agrigento,1.39,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,4,279,387,2024-10-06,197,217,NaN,23,23,NaN,37.319422,13.617396


In [15]:
df_filtered['scheda_istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s='+df_filtered['id']+'">Vai alla scheda istituto.</a>"'

df_filtered.head()

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_26371/2592475969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['scheda_istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s='+df_filtered['id']+'">Vai alla scheda istituto.</a>"'


,id,nome,tasso_affollamento,indirizzo,tipo,posti_regolamentari,posti_non_disponibili,posti_reali,posti_occupati,posti_aggiornati_al,personale_polizia_previsti,personale_polizia_effettivi,personale_polizia_aggiornato_a,personale_amministrativi_previsti,personale_amministrativi_effettivi,personale_amministrativo_aggiornato_al,latitudine,longitude,scheda_istituto
0,MII152284,Genova Marassi,1.26,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,676,2024-10-06,336,357,NaN,35,26,NaN,44.417913,8.951100,"<a href=""https://www.giustizia.it/giustizia/pa..."
1,MII157783,Alba,0.78,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,38,2024-10-06,107,88,NaN,14,10,NaN,44.691542,8.025636,"<a href=""https://www.giustizia.it/giustizia/pa..."
2,MII158895,Milano San Vittore,2.28,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,700,254,446,1016,2024-10-06,655,583,NaN,45,35,NaN,45.461598,9.166399,"<a href=""https://www.giustizia.it/giustizia/pa..."
3,MII158901,Belluno,1.28,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,105,2024-10-06,85,81,NaN,14,8,NaN,46.139738,12.219297,"<a href=""https://www.giustizia.it/giustizia/pa..."
4,MII158910,Agrigento,1.39,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,4,279,387,2024-10-06,197,217,NaN,23,23,NaN,37.319422,13.617396,"<a href=""https://www.giustizia.it/giustizia/pa..."


In [16]:
# Saving csv
df_filtered.to_csv('../outputs/viz/institutes_most_recent.csv', index=False, encoding='UTF-8-sig')

## 1. Totals

In [17]:
grouped_df = df.groupby('posti_aggiornati_al').sum(numeric_only=True).reset_index()
grouped_df

,posti_aggiornati_al,posti_regolamentari,posti_non_disponibili,posti_occupati,personale_polizia_effettivi,personale_polizia_previsti,personale_amministrativi_effettivi,personale_amministrativi_previsti,personale_educatori_effettivi,personale_educatori_previsti
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001


In [18]:
grouped_df['posti_reali'] = (grouped_df['posti_regolamentari'] - grouped_df['posti_non_disponibili']).round(2)

grouped_df['tasso_sovraffollamento'] = (grouped_df['posti_occupati'] / grouped_df['posti_reali']).round(2)

grouped_df.head()


,posti_aggiornati_al,posti_regolamentari,posti_non_disponibili,posti_occupati,personale_polizia_effettivi,personale_polizia_previsti,personale_amministrativi_effettivi,personale_amministrativi_previsti,personale_educatori_effettivi,personale_educatori_previsti,posti_reali,tasso_sovraffollamento
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,46812,1.32
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,46812,1.32


In [19]:
df_summed.to_csv('../outputs/viz/institutes_totals.csv', index=False)

NameError: name 'df_summed' is not defined